In [1]:
import pandas as pd
from os.path import join,isfile
from os import listdir
import os
import dask
import bookquery
from SPARQLWrapper import SPARQLWrapper, JSON
from fake_useragent import UserAgent
ua = UserAgent()

def execute_query(qquery, entity_id):# Работа с запросами
    
    """Работа с запросами"""
    
    sparql = SPARQLWrapper("https://query.wikidata.org/sparql",agent = ua.random)
    sparql.setQuery(qquery.format(entity_id = entity_id))
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    return results['results']['bindings']

def entities_sparqul(results):
    
    """Обработка результата"""
    
        res = []
        for result in results:# Обработка вернувшегося запрсов
            res.append(result['propLabel']['value'])# Создаем массив словарей
        return res
    
def selector_for_predicates(entity_id):
    
    """Функция получения всех предикатов"""
    
    raw_results= execute_query(bookquery.finding_ID, entity_id)
    res = entities_sparqul(raw_results)
    return res

In [9]:
res = selector_for_predicates('Q19847637')

In [10]:
wrong_predicates = ['код LCNAF' , 'идентификатор Structurae' , 'это частный случай понятия' , 'код GeoNames' , 'не путать с', 
'код аэропорта ФАА','код GNIS' , 'код Freebase','изображение','категория на Викискладе','указания, как добраться','код VIAF',
'код LCNAF','категория карт на Викискладе','категория объединяет темы','код BabelNet' , 'баннер Викигида',
'в точности соответствует','изображённый объект','основная тема','Enciclopédia Itaú Cultural ID','ID отношения в OpenStreetMap',
"Who's on First ID" , 'категория объединяет темы' , 'идентификатор объекта в картах Google' , 'код темы Quora',
'изображение интерьера','часть от','категория Викисклада для интерьера элемента' , 'начало пути' , 'смежное здание' , 'Wi-Fi'
,'заменил' , ' заменён на']


wrong_predicates.extend(res)

correct_predicates = ['код аэропорта ИКАО','код аэропорта ИАТА','код ОКАТО','код ОКТМО']

wrong_predicates = list(set(wrong_predicates) ^  set(correct_predicates))
def filter_func(x):
    
    """Функция для фильтрации"""
        return x['prop_name'].max() not in wrong_predicates

In [11]:
def counter_for_files(path):
    
    """Подсчет файлов в директории"""
    
    onlyfiles = [join(path, f) for f in listdir(path) if isfile(join(path, f))]
    return onlyfiles

def predicate_selection(file):
    
    """Фильтрация предикатов в файлах"""
    
    label  = file.split('\\')[-1].split('.')[0].split('_')[2]
    print(f'Файл {label} начал обрабатываться')
    df= pd.read_csv(file,encoding='utf8',sep = ';')
    df = df.groupby('prop_name').filter(filter_func)
    df.to_csv(file , encoding='utf-8-sig',index = False,sep = ';')
    print(f'Файл {label} закончил обрабатываться')

In [13]:
%%time
curr_dirrectory =r"C:\Users\Алексей Кушнер\Sberbank\Geography\Information\2\analyzing\aggregated"
ress = []
for filename in counter_for_files(curr_dirrectory):
    ress.append(dask.delayed(predicate_selection)(filename))
result = dask.compute(*ress)

Файл Q3240715 начал обрабатываться
Файл Q39816 начал обрабатыватьсяФайл Q184122 начал обрабатываться
Файл Q835714 начал обрабатываться

Файл Q83471 начал обрабатываться
Файл Q831740 начал обрабатыватьсяФайл Q1402592 начал обрабатыватьсяФайл Q62447 начал обрабатываться
Файл Q150784 начал обрабатыватьсяФайл Q483453 начал обрабатываться
Файл Q35509 начал обрабатываться



Файл Q7930989 начал обрабатыватьсяФайл Q12280 начал обрабатыватьсяФайл Q319714 начал обрабатыватьсяФайл Q33506 начал обрабатываться


Файл Q35657 начал обрабатываться

Файл Q1233637 начал обрабатыватьсяФайл Q37901 начал обрабатыватьсяФайл Q211748 начал обрабатываться


Файл Q1248784 начал обрабатываться
Файл Q35666 начал обрабатываться
Файл Q1210950 начал обрабатываться
Файл Q46831 начал обрабатываться
Файл Q6256 начал обрабатываться
Файл Q319714 закончил обрабатываться
Файл Q55488 начал обрабатываться
Файл Q3240715 закончил обрабатываться
Файл Q12284 начал обрабатываться
Файл Q83471 закончил обрабатываться
Файл Q3253281